In [1]:
import pandas as pd
import numpy as np
# import requests
from bs4 import BeautifulSoup
import re

In [2]:
# URL = "data/Micro0 Gonzalo septiembre2021.html"
# page = requests.get(URL)
# print(page.text)

# 1. Get data from .html

The storage structure is as follows (JSON compatibl-ish):
```
{micro_name: [
      {"workout_desc": "Pierna",
       "date_workout": session1_date,
       "exercises": [
              {"nombre": "sentadilla",
               "series": 4, 
               ...},
              {"nombre": "peso muerto",
               "series": 4}, 
              ...
              ]
      },
      {"workout_desc": "Torso",
       "date_workout": session2_date,
       "exercises": [
              {"nombre": "dominadas",
               "series": 4, 
               ...},
              {"nombre": "press banca",
               "series": 4}, 
              ...
              ]
      },
      ...
] }
```

In [3]:
def get_data_from_html(file):
# "data/Micro0 Gonzalo septiembre2021.html"  

    with open(file) as html_file:
        soup = BeautifulSoup(html_file, "html.parser")

        sessions_list = []
        sessions = soup.find_all("div", class_="dia")
        for session in sessions:
            session_dict = {}

            workout_desc = session.find("div", class_="titulo").text.lower()
            session_dict["workout_desc"] = workout_desc

            day = int(session.find("div", id="dia").text)
            month = int(session.find("div", id="mes").text)
            year = int(session.find("div", id="anyo").text)
            date = pd.Timestamp(day=day, month=month, year=year).date()
            session_dict["date_workout"] = date
            
            exercises_list = []
            exercises = session.find("div", class_="cuerpo-boxdia").find_all("div", re.compile("ejercicio.*"))
            for exercise in exercises:
                exercise_dict = {}
                for element in exercise.find_all("div"):
                    exercise_dict[element.attrs['class'][0]] = element.text
                exercises_list.append(exercise_dict)
            session_dict["exercises"] = exercises_list

            sessions_list.append(session_dict)

        block_name = soup.find(id="microciclo").text
        micro_dict = {block_name: sessions_list}

    return micro_dict

get_data_from_html("data/Micro0 Gonzalo septiembre2021.html")

{'Micro 0 Gonzalo septiembre2021': [{'workout_desc': 'pierna ',
   'date_workout': datetime.date(2021, 9, 6),
   'exercises': [{'nombre': 'Sentadilla',
     'series': '4',
     'cargas': '0 %',
     'kilos': '70 Kg',
     'repeticiones': '10',
     'rpe': '8',
     'descanso': '2 min'},
    {'nombre': 'Peso muerto',
     'series': '4',
     'cargas': '0 %',
     'kilos': '100 Kg',
     'repeticiones': '5',
     'rpe': '5',
     'descanso': '2 min'},
    {'nombre': 'Extensión de cuadriceps',
     'series': '4',
     'cargas': '0 %',
     'kilos': '50 Kg',
     'repeticiones': '10',
     'rpe': '8',
     'descanso': '1 min'},
    {'nombre': 'Femoral en maquina',
     'series': '4',
     'cargas': '0 %',
     'kilos': '20 Kg',
     'repeticiones': '10',
     'rpe': '7',
     'descanso': '1 min'},
    {'nombre': 'Abdominales crunch (Solo 15º) ',
     'series': '5',
     'cargas': '70 %',
     'kilos': '2 Kg',
     'repeticiones': '15',
     'rpe': '7',
     'descanso': '1 min'}]},
  {'work

# 2. Curate data

In [4]:
SESSION_COL_NAMES = ["Ejercicio", "Series", "Cargas (%)", 
                        "Kilos", "Repeticiones", "RPE", "Descanso (min)"]

def curate_exercises_data(exercises:list, col_names:list):

    df_exercises = pd.DataFrame(exercises)
    # # Data quality criteria for workout session table
    # Standard names (only if matching length, else keep originals)
    if len(df_exercises.columns) == len(col_names):
        df_exercises.columns = col_names
    # Lowercase exercise names
    df_exercises.iloc[:, 0] = df_exercises.iloc[:, 0].str.lower()
    # Convert to numbers
    df_exercises.iloc[:, 1] = df_exercises.iloc[:, 1].str.extract("(\d+)").astype("int")
    df_exercises.iloc[:, 2] = df_exercises.iloc[:, 2].str.extract("(\d+)").astype("float")
    df_exercises.iloc[:, 3] = df_exercises.iloc[:, 3].str.extract("(\d+)").astype("float")
    df_exercises.iloc[:, 4] = df_exercises.iloc[:, 4].str.extract("(\d+)").astype("int")
    df_exercises.iloc[:, 5] = df_exercises.iloc[:, 5].str.extract("(\d+)").astype("int")
    df_exercises.iloc[:, 6] = df_exercises.iloc[:, 6].str.extract("(\d+)").astype("float")
    # In "Cargas (%)" and "Descanso (min)" change 0 --> NULL
    df_exercises.iloc[:, 2] = df_exercises.iloc[:, 2].replace(0.0, np.nan)
    df_exercises.iloc[:, 6] = df_exercises.iloc[:, 6].replace(0.0, np.nan)

    return df_exercises


block_dict = get_data_from_html("data/Micro0 Gonzalo septiembre2021.html")
for block_name, workouts_list in block_dict.items():
    print(block_name)
    print("-"*30)
    for workout in workouts_list:
        print(f"{workout['date_workout']} - {workout['workout_desc']}")
        df = curate_exercises_data(workout["exercises"], SESSION_COL_NAMES)
        display(df)
        # data_ex = df["Ejercicio"].unique()
        # db_ex = [i[0] for i in session.query(Exercise.exercise_desc).all()]
        # print(set(data_ex))
        # print(list(set(data_ex) - set(db_ex)))

Micro 0 Gonzalo septiembre2021
------------------------------
2021-09-06 - pierna 


,Ejercicio,Series,Cargas (%),Kilos,Repeticiones,RPE,Descanso (min)
0,sentadilla,4,NaN,70.0,10,8,2.0
1,peso muerto,4,NaN,100.0,5,5,2.0
2,extensión de cuadriceps,4,NaN,50.0,10,8,1.0
3,femoral en maquina,4,NaN,20.0,10,7,1.0
4,abdominales crunch (solo 15º),5,70.0,2.0,15,7,1.0


2021-09-07 - torso


,Ejercicio,Series,Cargas (%),Kilos,Repeticiones,RPE,Descanso (min)
0,press de banca,4,60.0,60.0,10,7,2.0
1,press plano mancuernas,4,70.0,25.0,10,7,2.0
2,dominadas libres,4,70.0,80.0,8,6,2.0
3,remo con barra,4,50.0,50.0,10,5,1.0
4,press francés barra olímpica,5,70.0,20.0,10,5,1.0


2021-09-09 - acc pierna pst


,Ejercicio,Series,Cargas (%),Kilos,Repeticiones,RPE,Descanso (min)
0,peso muerto rumano,4,NaN,50.0,12,7,1.0
1,peso muerto rumano con mancuernas,4,NaN,20.0,10,8,1.0
2,hip thrust,4,NaN,100.0,8,6,2.0
3,sentadilla búlgara (2 mancuernas de 10),4,NaN,10.0,8,7,1.0
4,gemelo en multipower,5,NaN,60.0,12,6,1.0


2021-09-11 - acc torso


,Ejercicio,Series,Cargas (%),Kilos,Repeticiones,RPE,Descanso (min)
0,press inclinado,4,50.0,50.0,12,6,2.0
1,press militar,4,70.0,30.0,12,8,2.0
2,press militar mancuernas sentado con respaldo,4,70.0,10.0,12,6,1.0
3,jalón al pecho,4,70.0,40.0,12,7,1.0
4,curl de biceps barra olímpica,4,70.0,20.0,12,8,1.0
5,curl de biceps mancuernas,4,70.0,10.0,10,7,1.0
6,face pull,5,70.0,10.0,15,8,1.0
7,rotación externa de hombro en polea,5,70.0,5.0,15,9,1.0


In [5]:
# for index, row in df.iterrows():
#     for set in range(row["Series"]):
#         set_id = set + 1
#         print(row.drop("Series")["Ejercicio"], row.drop("Series")["Cargas (%)"], set_id)

# session.query(Exercise.exercise_id).filter_by(exercise_desc="press banca").scalar()

In [6]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from models import Program, Block, Workout, Workout_set, Exercise

engine = create_engine(f"sqlite:///data/db/gym_database.db")
Session = sessionmaker(bind=engine)
session = Session()

In [8]:
# Adding block of program from personal coach html file
def add_block(session, source_file=None, program:str=None):
    
    COL_NAMES = ["Ejercicio", "Series", "Cargas (%)", 
                 "Kilos", "Repeticiones", "RPE", "Descanso (min)"]

    # Check if program description already exists
    if program:
        program = (
            session.query(Program)
            .filter(Program.program_desc==program)
            .one_or_none()
        )
        program_id = program.program_id
    # If not (or not provided), create new generic program
    if program is None:
        program = Program()
        session.add(program)
        # If new, the program_id will be last element added
        program_id = session.query(Program).count()

    block_dict = get_data_from_html(source_file)

    for block_name, workouts_list in block_dict.items():
        # Check if block already exist (matching both name and program)
        block = (
            session.query(Block)
            .filter(Block.block_desc==block_name, Block.program_id==program_id)
            .one_or_none()
        )
        if block:
            block_id = block.block_id
        # If not, create it
        if block is None:
            block = Block(block_desc=block_name, program_id=program_id)
            session.add(block)
            # If new, the block_id will be last element added
            block_id = session.query(Block).count() 

            # The loading of workouts and sets is only done if block doesn't exist
            # If block already exists we could branch an update function inside
            # (inside this add_block() or outside as a different call)
            for wod in workouts_list:
                workout = Workout(workout_desc=wod["workout_desc"], 
                                  block_id=block_id, 
                                  date_workout=wod["date_workout"])
                session.add(workout)
                workout_id = session.query(Workout).count() 
                df_exercises = curate_exercises_data(wod["exercises"], COL_NAMES)
                
                # If exercise is not in Exercise lookup table, add it previously
                data_ex = df_exercises["Ejercicio"].unique()
                db_ex = [i[0] for i in session.query(Exercise.exercise_desc).all()]
                for new_exercise in list(set(data_ex) - set(db_ex)):
                    exercise = Exercise(exercise_desc=new_exercise)
                    session.add(exercise)
                
                # Then, insert row by row the results (exploding for as many series
                # per exercise there are)
                for index, row in df_exercises.iterrows():
                    for wod_set in range(row["Series"]):
                        set_id = wod_set + 1
                        workout_set = Workout_set(
                            workout_id=workout_id,
                            exercise_id=session.query(Exercise.exercise_id)
                                               .filter_by(exercise_desc=row["Ejercicio"])
                                               .scalar(),
                            set_id=set_id,
                            no_reps=row["Repeticiones"],
                            weight=row["Kilos"],
                            perc_rm=row["Cargas (%)"] if row["Cargas (%)"] != np.nan else None,
                            max_rpe=row["RPE"],
                            rest_min=row["Descanso (min)"] if row["Descanso (min)"] != np.nan else None
                        )
                        session.add(workout_set)

    pass

add_block(session, "data/Micro0 Gonzalo septiembre2021.html")

In [10]:
# session.rollback()

# session.query(Program).all()
# session.query(Program).order_by(Program.program_id.desc()).first().program_id
# session.query(Program).count()

# session.query(Block).filter(Block.program_id==4).first().program
# session.query(Block).all()

# session.query(Exercise).all()

# session.query(Workout).all()

# session.query(Workout_set).all()
session.query(Workout_set.perc_rm).all()

[(92.5,),
 (92.5,),
 (None,),
 (None,),
 (None,),
 (None,),
 (None,),
 (None,),
 (None,),
 (None,),
 (None,),
 (None,),
 (None,),
 (None,),
 (None,),
 (None,),
 (None,),
 (None,),
 (70.0,),
 (70.0,),
 (70.0,),
 (70.0,),
 (70.0,),
 (60.0,),
 (60.0,),
 (60.0,),
 (60.0,),
 (70.0,),
 (70.0,),
 (70.0,),
 (70.0,),
 (70.0,),
 (70.0,),
 (70.0,),
 (70.0,),
 (50.0,),
 (50.0,),
 (50.0,),
 (50.0,),
 (70.0,),
 (70.0,),
 (70.0,),
 (70.0,),
 (70.0,),
 (None,),
 (None,),
 (None,),
 (None,),
 (None,),
 (None,),
 (None,),
 (None,),
 (None,),
 (None,),
 (None,),
 (None,),
 (None,),
 (None,),
 (None,),
 (None,),
 (None,),
 (None,),
 (None,),
 (None,),
 (None,),
 (50.0,),
 (50.0,),
 (50.0,),
 (50.0,),
 (70.0,),
 (70.0,),
 (70.0,),
 (70.0,),
 (70.0,),
 (70.0,),
 (70.0,),
 (70.0,),
 (70.0,),
 (70.0,),
 (70.0,),
 (70.0,),
 (70.0,),
 (70.0,),
 (70.0,),
 (70.0,),
 (70.0,),
 (70.0,),
 (70.0,),
 (70.0,),
 (70.0,),
 (70.0,),
 (70.0,),
 (70.0,),
 (70.0,),
 (70.0,),
 (70.0,),
 (70.0,),
 (70.0,),
 (70.0,)]

In [12]:
session.close()
engine.dispose()

In [13]:
session